<a href="https://colab.research.google.com/github/Greta-gerasimov/NLP_1/blob/forecast_telegram_bot/simple_forecat_telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install python-telegram-bot --upgrade

In [ ]:
!pip install requests

In [15]:
import logging
from telegram import Update, Bot
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import requests

import nest_asyncio
nest_asyncio.apply()


In [ ]:
import logging
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
from telegram import Update
import requests

# Вставьте сюда ваш токен телеграм-бота
TELEGRAM_BOT_TOKEN = '6616113018:AAG4tJWZd-062P_xuwWwFp4091GAVHkb5rc'
# Вставьте сюда ваш API-ключ OpenWeatherMap
OPENWEATHER_API_KEY = 'a527d39537f724fb4f876980ba4256c7'
# Базовый URL для запроса погоды
WEATHER_API_URL = "http://api.openweathermap.org/data/2.5/weather"

# Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Команда start
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text('Привет! Напиши название города, и я предскажу тебе погоду.')

# Функция для получения погоды
async def get_weather(city: str) -> str:
    try:
        params = {
            'q': city,
            'appid': OPENWEATHER_API_KEY,
            'units': 'metric',
            'lang': 'ru'
        }
        response = requests.get(WEATHER_API_URL, params=params)
        response.raise_for_status()
        weather_data = response.json()

        city_name = weather_data['name']
        country = weather_data['sys']['country']
        temp = weather_data['main']['temp']
        weather_description = weather_data['weather'][0]['description']
        return (f"Погода в городе {city_name}, {country}: {temp}°C, "
                f"состояние: {weather_description}")
    except Exception as e:
        logger.error(f"Произошла ошибка при получении погоды: {e}")
        return "Произошла ошибка при получении данных о погоде."

# Обработка сообщения с названием города
async def handle_weather(update: Update, context: CallbackContext) -> None:
    city = update.message.text
    weather = await get_weather(city)
    await update.message.reply_text(weather)

# Главная функция, в которой запускается бот
async def main() -> None:
    # Создание приложения бота с определенным токеном
    application = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

    # Добавление команды /start
    start_handler = CommandHandler('start', start)
    application.add_handler(start_handler)

    # Добавление обработчика текстовых сообщений для запроса погоды
    weather_handler = MessageHandler(filters.TEXT & ~filters.COMMAND, handle_weather)
    application.add_handler(weather_handler)

    # Запускаем бота
    await application.run_polling()

if __name__ == '__main__':
    # Запуск главной функции
    import asyncio
    asyncio.run(main())